In [1]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

import re
import pandas as pd
import numpy as np
import pickle
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity

from matplotlib import pyplot as plt

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\BILAL\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import sys
sys. version

'3.10.9 | packaged by Anaconda, Inc. | (main, Mar  1 2023, 18:18:15) [MSC v.1916 64 bit (AMD64)]'

In [3]:
icd_categories_description = pd.read_csv('categories.csv')

icd_categories_description

,icd_code,description
0,A00,Cholera
1,A01,Typhoid and paratyphoid fevers
2,A010,Typhoid fever
3,A011,Paratyphoid fever A
4,A012,Paratyphoid fever B
...,...,...
24623,Z99,"Dependence on enabling machines and devices, n..."
24624,Z991,Dependence on respirator
24625,Z992,Dependence on renal dialysis
24626,Z993,Dependence on wheelchair


In [4]:
# Define the reformat function
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [5]:
icd_categories_description = icd_categories_description[icd_categories_description['icd_code'].astype(str).apply(len) <= 3]

icd_categories_description['icd_code'] = icd_categories_description['icd_code'].apply(lambda x: reformat(x))
icd_categories_description.reset_index(drop = True)

C:\Users\BILAL\AppData\Local\Temp\ipykernel_19552\242373274.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  icd_categories_description['icd_code'] = icd_categories_description['icd_code'].apply(lambda x: reformat(x))


,icd_code,description
0,A00.,Cholera
1,A01.,Typhoid and paratyphoid fevers
2,A02.,Other salmonella infections
3,A03.,Shigellosis
4,A04.,Other bacterial intestinal infections
...,...,...
1905,Z95.,Presence of cardiac and vascular implants and ...
1906,Z96.,Presence of other functional implants
1907,Z97.,Presence of other devices
1908,Z98.,Other postprocedural states


In [6]:
icd_categories_description.index = icd_categories_description['icd_code']
icd_categories_description

,icd_code,description
icd_code,,
A00.,A00.,Cholera
A01.,A01.,Typhoid and paratyphoid fevers
A02.,A02.,Other salmonella infections
A03.,A03.,Shigellosis
A04.,A04.,Other bacterial intestinal infections
...,...,...
Z95.,Z95.,Presence of cardiac and vascular implants and ...
Z96.,Z96.,Presence of other functional implants
Z97.,Z97.,Presence of other devices


In [31]:
from transformers import AutoTokenizer, AutoModel, AutoModelForTokenClassification
import torch

# Mean Pooling - Take attention mask into account for correct averaging
def meanpooling(output, mask):
    embeddings = output[0] # First element of model_output contains all token embeddings
    mask = mask.unsqueeze(-1).expand(embeddings.size()).float()
    return torch.sum(embeddings * mask, 1) / torch.clamp(mask.sum(1), min=1e-9)


In [9]:
# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained("NeuML/pubmedbert-base-embeddings")
model = AutoModel.from_pretrained("NeuML/pubmedbert-base-embeddings")

def create_description_embeddings(description):


    # Tokenize sentences
    inputs = tokenizer(description, padding=True, truncation=True, return_tensors='pt')

    # Compute token embeddings
    with torch.no_grad():
        output = model(**inputs)

    # Perform pooling. In this case, mean pooling.
    embeddings = meanpooling(output, inputs['attention_mask'])
    
    return embeddings

In [11]:
testing_notes_with_smallest_size = pd.read_csv('1000_testing_notes_with_smallest_size.csv')

In [12]:
testing_notes_with_smallest_size.drop(columns = ['Unnamed: 0'], inplace = True)

In [13]:
icd_codes_with_smallest_size_notes = pd.read_csv('icd_codes_for_1000_testing_notes_with_smallest_size.csv')

In [14]:
icd_codes_with_smallest_size_notes.drop(columns = ['Unnamed: 0'], inplace = True)

In [15]:
final_notes_with_icd_codes = pd.merge(testing_notes_with_smallest_size, icd_codes_with_smallest_size_notes, on='hadm_id', how='left')
final_notes_with_icd_codes

,hadm_id,text,text_length,icd_code
0,20019282,\nName: ___ Unit No: ___\n...,2480,S82.
1,20019282,\nName: ___ Unit No: ___\n...,2480,Z72.
2,20019282,\nName: ___ Unit No: ___\n...,2480,W10.
3,20019282,\nName: ___ Unit No: ___\n...,2480,Y92.
4,20022343,\nName: ___ Unit No: _...,3001,O60.
...,...,...,...,...
8580,29990058,\nName: ___ Unit No: ___\n...,3051,Z30.
8581,29990058,\nName: ___ Unit No: ___\n...,3051,Z23.
8582,29990058,\nName: ___ Unit No: ___\n...,3051,Z86.
8583,29990058,\nName: ___ Unit No: ___\n...,3051,K21.


In [16]:
medical_stopwords = stopwords.words("english")


medical_stopwords.extend(['speaking', 'none', 'time', 'flush'])

In [17]:
def process_clinical_note(clinical_note):
    # Define the sections to remove
    sections_to_remove = [
        "Name:",
        "Unit No:",
        "Admission Date:",
        "Discharge Date:",
        "Date of Birth:",
        "Sex:",
        "Service:",
        "Allergies:",
        "Attending:",
        "Past Medical History:",
        "Social History:",
        "Family History:",
        "Vitals:",
        "Pertinent Results:",
        "Medications on Admission:",
        "Discharge Medications:",
        "Discharge Disposition:",
        "Discharge Condition:",
        "Discharge Instructions:",
        "Followup Instructions:"
    ]

    # Split the clinical note into lines
    lines = clinical_note.split('\n')

    # Initialize the processed note
    processed_note = []

    # Flag to exclude lines within unwanted sections
    exclude_section = False

    # Iterate through the lines and filter unwanted sections
    for line in lines:
        if any(section in line for section in sections_to_remove):
            exclude_section = True
        elif line.strip() == "":
            # Empty lines separate sections, so reset the flag
            exclude_section = False

        if not exclude_section:
            processed_note.append(line)

    # Join the lines to create the final note
    final_note = '\n '.join(processed_note)
    
    sections_to_remove = [
        r'chief complaint',
        r'history of present illness',
        r'Major Surgical or Invasive Procedure',
        r'physical exam',
        r'brief hospital course',
        r'Discharge',
        
        r'completed by',
    ]
    
    for pattern in sections_to_remove:
        final_note = re.sub(pattern, '', final_note, flags=re.IGNORECASE)

    # Define patterns to identify negations
    negation_patterns = [
        r'no\s+\w+',
        r'not\s+\w+',
        r'did\s+not\s+have\s+\w+'
    ]
    
    # Filter out sentences with negations
    sentences = [sentence for sentence in final_note.split('\n') if not any(re.search(pattern, sentence, re.IGNORECASE) for pattern in negation_patterns)]

    # Remove keys and special characters
    cleaned_note = re.sub(r'\w+:', '', '\n'.join(sentences), flags=re.IGNORECASE)  # Remove keys (case-insensitive)
    cleaned_note = re.sub(r'[^a-zA-Z\s]', '', cleaned_note)  # Remove special characters
    # Tokenize the note into sentences based on '\n'
    sentences = [sentence.strip() for sentence in cleaned_note.split('\n') if sentence.strip()]

    # Remove stop words and empty sentences
    sentences = [
        ' '.join(word for word in sentence.split() if word.lower() not in medical_stopwords)
        for sentence in sentences
    ]
    sentences = [item for item in sentences if item != '']
    
    final_text = '. '.join(sentence for sentence in sentences)
    return final_text

In [18]:
import json
import pinecone


from collections import Counter
import numpy as np
from functools import partial
import string
import datasets

import re
import matplotlib.pyplot as plt


In [647]:
## Storing API keys
pinecone_api_key = "24c6f012-a062-4e67-b4cd-b9ab9b555cd0"
pinecone_environment = "gcp-starter"


In [104]:
## Creating pinecone index

pinecone.init(api_key=pinecone_api_key, environment=pinecone_environment)
pinecone_index_name = "msproject"

## delete the index if it already exists
if pinecone_index_name in pinecone.list_indexes():
    pinecone.delete_index(pinecone_index_name)

# create the index
pinecone.create_index(name=pinecone_index_name, metric="cosine", shards=1, dimension=768)

In [648]:
# First we import pinecone. If you have not installed it, install it first by running pip install pinecone
pinecone.init(pinecone_api_key, environment=pinecone_environment)
pinecone_index_name = "msproject"
# connect to the newly created index
index = pinecone.Index(pinecone_index_name)

In [128]:
reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])
reference_embeddings = reference_embeddings.squeeze()
reference_embeddings = np.array([embedding.numpy() for embedding in reference_embeddings])

C:\Users\BILAL\AppData\Local\Temp\ipykernel_21812\2667019115.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])


In [129]:
reference_embeddings.shape

(1910, 768)

In [145]:
reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])
reference_embeddings = reference_embeddings.squeeze()
reference_embeddings = [embedding.tolist() for embedding in reference_embeddings]
shape = np.array(reference_embeddings).shape
shape

C:\Users\BILAL\AppData\Local\Temp\ipykernel_21812\2617819856.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  reference_embeddings = torch.stack([torch.tensor(embedding) for embedding in icd_categories_description['embeddings'].values])


(1910, 768)

In [140]:
icd_categories_description.description.tolist()

['Cholera',
 'Typhoid and paratyphoid fevers',
 'Other salmonella infections',
 'Shigellosis',
 'Other bacterial intestinal infections',
 'Other bacterial foodborne intoxications, not elsewhere classified',
 'Amebiasis',
 'Other protozoal intestinal diseases',
 'Viral and other specified intestinal infections',
 'Infectious gastroenteritis and colitis, unspecified',
 'Respiratory tuberculosis',
 'Tuberculosis of nervous system',
 'Tuberculosis of other organs',
 'Miliary tuberculosis',
 'Plague',
 'Tularemia',
 'Anthrax',
 'Brucellosis',
 'Glanders and melioidosis',
 'Rat-bite fevers',
 'Erysipeloid',
 'Leptospirosis',
 'Other zoonotic bacterial diseases, not elsewhere classified',
 "Leprosy [Hansen's disease]",
 'Infection due to other mycobacteria',
 'Listeriosis',
 'Tetanus neonatorum',
 'Obstetrical tetanus',
 'Other tetanus',
 'Diphtheria',
 'Whooping cough',
 'Scarlet fever',
 'Meningococcal infection',
 'Streptococcal sepsis',
 'Other sepsis',
 'Actinomycosis',
 'Nocardiosis',
 

In [142]:
meta = []
for i in range(len(icd_categories_description)):
    
    temp = {}
    temp['icd_10_code'] = icd_categories_description['icd_code'][i]
    temp['description'] = icd_categories_description['description'][i]
    meta.append(temp)
len(meta)

1910

In [134]:
meta[0]

{'icd_10_code': 'A00.', 'description': 'Cholera'}

In [148]:
batch_size = 100

## ids
ids = [str(i) for i in range(len(icd_categories_description))]

# create list of (id, vector, metadata) tuples to be upserted
to_upsert = list(zip(ids, reference_embeddings, meta))
for i in range(0, shape[0], batch_size):
    i_end = min(i+batch_size, shape[0])
    index.upsert(vectors=to_upsert[i:i_end])

In [156]:
df_output = pd.DataFrame()

In [50]:
sentences = ['Left ankle fracture',
 'ORIF left ankle',
 'yr old man fall stairs holding lumbar',
 'construction',
 'job resulting immediate L ankle pain ED ankle fracture',
 'surgery performed',
 'AOXO',
 'splint intact positive CSM',
 'Taken operating room underwent surgical fixation ankle',
 'well controlled Cleared go home without services',
 'left ankle fracture sp ORIF']

In [54]:
fun = '. '.join(sentences)
fun

'Left ankle fracture. ORIF left ankle. yr old man fall stairs holding lumbar. construction. job resulting immediate L ankle pain ED ankle fracture. surgery performed. AOXO. splint intact positive CSM. Taken operating room underwent surgical fixation ankle. well controlled Cleared go home without services. left ankle fracture sp ORIF'

In [217]:
sentences = ['Stroke',
 'yr F w hx ESRD HD HTN DMII prior stroke',
 'w residual mild aphasia transferred OSH due',
 'new onset R sided weakness profound aphasia concerning',
 'stroke Exam consistent L MCA syndrome w pt seen',
 'severe expressive aphasia L gaze preference R',
 'sided weakness NIHSS OSH tPA given OSH',
 'NIHSS Etiology could potentially',
 'cardioembolic given recurrent stroke proximal vasculature',
 'appearing clear affected side hour head CT showed',
 'large L MCA',
 'expected entire territory infarct',
 'appeared relatively good distal reconstitution',
 'fetal PCA L appeared supplying MCA',
 'infarct quite large affecting',
 'cortical motor areas subcortical fibers would expect',
 'remain hemiplegic R exam developed',
 'global aphasia hemiplegia R Goals care',
 'discussed family made comfort measures',
 'passed away',
 'ESRD received HD renal consulted',
 'management Per family request discontinued',
 'goals shifted comfort measures',
 'yeast peritoneal fluid yeast grew peritoneal fluid',
 'culture started fluconazole ID',
 'consulted discontinued made CMO',
 'Stroke'
, '']

In [22]:
notes = create_description_embeddings(sentences)
notes_embeddings = [embedding.tolist() for embedding in notes]

In [23]:
sentences_list = []
icd_10_code_list = []
description_list = []
score_list = []

for i in range(0, len(notes_embeddings)):
    similar = index.query(notes_embeddings[i], top_k=1, include_metadata=True)
    
    sentences_list.append(sentences[i])
    
    if similar['matches']:
        match = similar['matches'][0]
        icd_10_code_list.append(match['metadata']['icd_10_code'])
        description_list.append(match['metadata']['description'])
        score_list.append(match['score'])
    else:
        icd_10_code_list.append(None)
        description_list.append(None)
        score_list.append(None)

# Create a DataFrame
df_output = pd.DataFrame({
    'Sentences': sentences_list,
    'ICD_10_Code': icd_10_code_list,
    'Description': description_list,
    'Score': score_list
})



In [24]:
df_output[df_output.Score >=0.0]

,Sentences,ICD_10_Code,Description,Score
0,Left ankle fracture,S82.,"Fracture of lower leg, including ankle",0.752511
1,ORIF left ankle,S97.,Crushing injury of ankle and foot,0.672783
2,yr old man fall stairs holding lumbar,W06.,Fall from bed,0.624991
3,construction,W13.,"Fall from, out of or through building or struc...",0.520577
4,job resulting immediate L ankle pain ED ankle ...,S97.,Crushing injury of ankle and foot,0.627952
5,surgery performed,Y83.,Surgical operation and other surgical procedur...,0.624054
6,AOXO,L01.,Impetigo,0.361797
7,splint intact positive CSM,S47.,Crushing injury of shoulder and upper arm,0.372391
8,Taken operating room underwent surgical fixati...,Z40.,Encounter for prophylactic surgery,0.530660
9,well controlled Cleared go home without services,Z53.,Persons encountering health services for speci...,0.420105


In [34]:
final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == 20053836].text[25]

' \nName:  ___                Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   F\n \nService: NEUROLOGY\n \nAllergies: \nCalcium Channel Blocking Agent Diltiazem Analogues\n \nAttending: ___\n \nChief Complaint:\nStroke\n \nMajor Surgical or Invasive Procedure:\nNone\n \nBrief Hospital Course:\n___ yr F w/ hx of ESRD on HD, HTN, DMII, and prior stroke in \n___ w/ residual mild aphasia who was transferred from OSH due \nto new onset R sided weakness and profound aphasia concerning \nfor stroke. Exam is consistent with a L MCA syndrome w/ pt seen\nto have severe expressive aphasia, L gaze preference, and R \nsided weakness. NIHSS at OSH was 22, tPA was given at OSH at \n___ on ___. NIHSS on ___ was 18. Etiology could potentially \ncardioembolic, given recurrent stroke and proximal vasculature\nappearing clear on affected side. Her 24 hour head CT showed \nlarge L MCA\ninfarct, with no hemorrhage. Based on the CTA would 

In [612]:


def get_icd_scores(sentences, index):
    result_list = []

    # Create embeddings for sentences
    notes_embeddings = create_description_embeddings(sentences)
    notes_embeddings = [embedding.tolist() for embedding in notes_embeddings]

    for i in range(len(sentences)):
        similar = index.query(notes_embeddings[i], top_k=1, include_metadata=True)

        if similar['matches']:
            match = similar['matches'][0]
            icd_code = match['metadata']['icd_10_code']
            score = match['score']
            result_list.append({'icd_code': icd_code, 'score': score})

    return result_list

In [26]:
import logging

# Configure logging to print to console
logging.basicConfig(level=logging.INFO)

# Assuming you have 'testing_notes_with_smallest_size' DataFrame and 'index' available



In [163]:
index

In [164]:
def get_icd_scores(sentences, index, row_count):
    result_list = []
    
    if len(sentences) != 0:
        # Create embeddings for sentences
        notes_embeddings = create_description_embeddings(sentences)
        notes_embeddings = [embedding.tolist() for embedding in notes_embeddings]

        for i, sentence in enumerate(sentences):
            try:
                similar = index.query(notes_embeddings[i], top_k=1, include_metadata=True)

                if similar['matches']:
                    match = similar['matches'][0]
                    icd_code = match['metadata']['icd_10_code']
                    score = match['score']
                    result_list.append({'icd_code': icd_code, 'score': score})

            except Exception as e:
                logging.error(f"Error processing row {row_count}: {str(e)}")

        # Print progress for every row processed
        logging.info(f"Processed {row_count} rows")

        return result_list
    
    else:
        return None


In [28]:
final_notes_with_icd_codes

,hadm_id,text,text_length,icd_code
0,20019282,\nName: ___ Unit No: ___\n...,2480,S82.
1,20019282,\nName: ___ Unit No: ___\n...,2480,Z72.
2,20019282,\nName: ___ Unit No: ___\n...,2480,W10.
3,20019282,\nName: ___ Unit No: ___\n...,2480,Y92.
4,20022343,\nName: ___ Unit No: _...,3001,O60.
...,...,...,...,...
8580,29990058,\nName: ___ Unit No: ___\n...,3051,Z30.
8581,29990058,\nName: ___ Unit No: ___\n...,3051,Z23.
8582,29990058,\nName: ___ Unit No: ___\n...,3051,Z86.
8583,29990058,\nName: ___ Unit No: ___\n...,3051,K21.


In [32]:
# Load pre-trained model and tokenizer
ner_tagging_tokenizer = AutoTokenizer.from_pretrained("medical-ner-proj/albert-medical-ner-proj")
ner_tagging_model = AutoModelForTokenClassification.from_pretrained("medical-ner-proj/albert-medical-ner-proj")



In [68]:
ner_classifier = pipeline('token-classification', model = ner_tagging_model, tokenizer = ner_tagging_tokenizer, aggregation_strategy = 'simple')


In [69]:
text = fun
predictions = ner_classifier(text)

In [70]:
fun

'Left ankle fracture. ORIF left ankle. yr old man fall stairs holding lumbar. construction. job resulting immediate L ankle pain ED ankle fracture. surgery performed. AOXO. splint intact positive CSM. Taken operating room underwent surgical fixation ankle. well controlled Cleared go home without services. left ankle fracture sp ORIF'

In [71]:
predictions

[{'entity_group': 'B_problem',
  'score': 0.9954737,
  'word': 'left',
  'start': 0,
  'end': 4},
 {'entity_group': 'I_problem',
  'score': 0.99743104,
  'word': 'ankle fracture',
  'start': 5,
  'end': 19},
 {'entity_group': 'B_problem',
  'score': 0.9942021,
  'word': 'or',
  'start': 21,
  'end': 23},
 {'entity_group': 'I_problem',
  'score': 0.99311036,
  'word': 'if left ankle',
  'start': 23,
  'end': 36},
 {'entity_group': 'B_problem',
  'score': 0.7172385,
  'word': 'immediate l',
  'start': 105,
  'end': 116},
 {'entity_group': 'I_problem',
  'score': 0.9110771,
  'word': 'ankle pain ed ankle fracture',
  'start': 117,
  'end': 145},
 {'entity_group': 'B_treatment',
  'score': 0.9050634,
  'word': 'surgery',
  'start': 147,
  'end': 154},
 {'entity_group': 'B_treatment',
  'score': 0.9190335,
  'word': 'a',
  'start': 166,
  'end': 167},
 {'entity_group': 'I_treatment',
  'score': 0.48994443,
  'word': 'oxo.',
  'start': 167,
  'end': 171},
 {'entity_group': 'B_treatment',
  '

In [106]:
def extract_entities(text):

    predictions = ner_classifier(text)

        # Extract entities without sentences
    entities = []
    current_entity = []

    for result in predictions:
        entity_label = result['entity_group']
        entity_text = result['word']

        if "problem" in entity_label.lower():  # Adjust the condition based on your specific labels
            if "B_" in entity_label:
                if current_entity:
                    entities.append(' '.join(current_entity))
                    current_entity = []

            current_entity.append(entity_text)

    # Append the last entity
    if current_entity:
        entities.append(' '.join(current_entity))

    return entities

In [133]:
# Example text
text = final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == 20053836].text[25]

# Get the list of entities
result_entities = extract_entities(text)
result_entities

['stroke',
 '',
 ' esrd',
 'prior stroke',
 'residual mild aphasia',
 'new onset r sided weakness',
 'profound aphasia',
 'stroke',
 'a l mca syndrome',
 'severe expressive aphasia',
 'l gaze preference',
 'r sided weakness',
 'cardio',
 'recurrent stroke vascula',
 'large l mca infarct',
 'he morrhage',
 'a fetal pca large',
 'he miplegic',
 'a global aphasia',
 'he miplegia',
 'yeast']

In [136]:
result_entities = extract_entities(processed_text)
result_entities


['stroke',
 '',
 'w h x',
 ' esrd',
 'hd htn',
 'dm ii',
 'prior stroke',
 'w residual mild aphasia',
 'new onset r sided weakness',
 'profound aphasia',
 'stroke',
 'consistent l mca syndrome',
 'severe expressive aphasia',
 'l gaze preference r',
 'side d weakness',
 'ni hss',
 'ni hss  ology',
 'cardio',
 'recurrent stroke proximal vasculature',
 'large l mca',
 'entire territory infarct a l',
 'in farct quite large tical motor areas',
 'he miplegic',
 'global aphasia',
 'he miplegia',
 'yeast peritoneal fluid yeast',
 'peri toneal fluid',
 'stroke']

In [137]:
get_icd_scores(result_entities, index)

[{'icd_code': 'I63.', 'score': 0.692511261},
 {'icd_code': 'A66.', 'score': 0.583028},
 {'icd_code': 'A66.', 'score': 0.419875354},
 {'icd_code': 'R80.', 'score': 0.549966216},
 {'icd_code': 'I11.', 'score': 0.546681523},
 {'icd_code': 'E11.', 'score': 0.549037695},
 {'icd_code': 'I69.', 'score': 0.440887362},
 {'icd_code': 'F70.', 'score': 0.421659589},
 {'icd_code': 'I40.', 'score': 0.410475343},
 {'icd_code': 'F73.', 'score': 0.599549592},
 {'icd_code': 'I63.', 'score': 0.692511261},
 {'icd_code': 'Z89.', 'score': 0.419373095},
 {'icd_code': 'F80.', 'score': 0.495987},
 {'icd_code': 'W15.', 'score': 0.39995563},
 {'icd_code': 'G56.', 'score': 0.448100537},
 {'icd_code': 'X18.', 'score': 0.372047275},
 {'icd_code': 'W15.', 'score': 0.408053607},
 {'icd_code': 'I42.', 'score': 0.645478308},
 {'icd_code': 'I66.', 'score': 0.59664768},
 {'icd_code': 'I66.', 'score': 0.386850685},
 {'icd_code': 'I63.', 'score': 0.506325722},
 {'icd_code': 'F95.', 'score': 0.452467948},
 {'icd_code': 'F65

In [108]:
final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == 25543633]

,hadm_id,text,text_length,icd_code
4722,25543633,\nName: ___ Unit No: ___\...,1417,R33.
4723,25543633,\nName: ___ Unit No: ___\...,1417,N17.
4724,25543633,\nName: ___ Unit No: ___\...,1417,E86.
4725,25543633,\nName: ___ Unit No: ___\...,1417,N40.
4726,25543633,\nName: ___ Unit No: ___\...,1417,E78.
4727,25543633,\nName: ___ Unit No: ___\...,1417,Z87.
4728,25543633,\nName: ___ Unit No: ___\...,1417,Z90.


In [109]:
text = testing_notes_with_smallest_size.iloc[571].text

In [110]:


# Get the list of entities
result_entities = extract_entities(text)
result_entities

['b ph',
 ' hld',
 'lip oma',
 'ex cision from r knee',
 'f',
 'stones',
 'gu mal ignancy']

In [119]:
get_icd_scores(result_entities, index)

[{'icd_code': 'W07.', 'score': 0.395925581},
 {'icd_code': 'A66.', 'score': 0.551725},
 {'icd_code': 'Q36.', 'score': 0.530958354},
 {'icd_code': 'W15.', 'score': 0.466602623},
 {'icd_code': 'W15.', 'score': 0.516161},
 {'icd_code': 'N20.', 'score': 0.65558672},
 {'icd_code': 'L01.', 'score': 0.48019433}]

In [112]:
testing_notes_with_smallest_size.iloc[571].text

' \nName:  ___                  Unit No:   ___\n \nAdmission Date:  ___              Discharge Date:   ___\n \nDate of Birth:  ___             Sex:   M\n \nService: EMERGENCY\n \nAllergies: \ncodeine\n \nAttending: ___.\n \nChief Complaint:\n \n \nMajor Surgical or Invasive Procedure:\n \n \nHistory of Present Illness:\n \n \nPast Medical History:\nPAST MEDICAL HISTORY: BPH, HLD\nPAST SURGICAL HISTORY: Lipoma excision from his neck, R knee \nmeniscus repair\n\n \nSocial History:\n___\nFamily History:\nNo FHx of stones or GU malignancy\n \nPhysical Exam:\n \n \nPertinent Results:\n___ 08:50PM BLOOD WBC-13.5* RBC-3.83* Hgb-11.3* Hct-34.1* \nMCV-89 MCH-29.5 MCHC-33.1 RDW-13.5 RDWSD-43.8 Plt ___\n___ 08:50PM BLOOD Neuts-75.4* Lymphs-12.0* Monos-10.0 \nEos-1.3 Baso-0.4 Im ___ AbsNeut-10.19*# AbsLymp-1.62 \nAbsMono-1.35* AbsEos-0.18 AbsBaso-0.06\n___ 12:00PM BLOOD Glucose-86 UreaN-21* Creat-1.6* Na-140 \nK-3.8 Cl-104 HCO3-25 AnGap-15\n___ 04:47AM BLOOD Glucose-91 UreaN-22* Creat-2.3*# Na-136

In [127]:
processed_text = process_clinical_note(final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == 20053836].text[25])
processed_text

'Stroke. yr F w hx ESRD HD HTN DMII prior stroke. w residual mild aphasia transferred OSH due. new onset R sided weakness profound aphasia concerning. stroke Exam consistent L MCA syndrome w pt seen. severe expressive aphasia L gaze preference R. sided weakness NIHSS OSH tPA given OSH. NIHSS Etiology could potentially. cardioembolic given recurrent stroke proximal vasculature. appearing clear affected side hour head CT showed. large L MCA. expected entire territory infarct. appeared relatively good distal reconstitution. fetal PCA L appeared supplying MCA. infarct quite large affecting. cortical motor areas subcortical fibers would expect. remain hemiplegic R exam developed. global aphasia hemiplegia R Goals care. discussed family made comfort measures. passed away. ESRD received HD renal consulted. management Per family request discontinued. goals shifted comfort measures. yeast peritoneal fluid yeast grew peritoneal fluid. culture started fluconazole ID. consulted discontinued made C

In [148]:
testing_notes_with_smallest_size['sentences'] = testing_notes_with_smallest_size['text'].apply(process_clinical_note)
testing_notes_with_smallest_size

,hadm_id,text,text_length,sentences
0,20019282,\nName: ___ Unit No: ___\n...,2480,Left ankle fracture. ORIF left ankle. yr old m...
1,20022343,\nName: ___ Unit No: _...,3001,Preterm labor. repeat low transverse cesarean ...
2,20025548,\nName: ___ Unit No: ___\n...,2961,contractions. SVD. Pt arrives co painful contr...
3,20027416,\nName: ___ Unit No: ___\n...,1059,neck cancer. direct laryngoscopy robotic hemig...
4,20028418,\nName: ___ Unit No: ___\n \...,1091,admission chemo. Pt tolerated desens without a...
...,...,...,...,...
1010,29980814,\nName: ___ Unit No: _...,3329,Postpartum incision check. partial opening pac...
1011,29980930,\nName: ___ Unit No: ___...,1094,elective chemotherapy. Pt presented elective c...
1012,29987299,\nName: ___ Unit No: ___\...,2595,attach. BLOOD WBC RBC Hgb Hct. MCV MCH MCHC RD...
1013,29988390,\nName: ___ Unit No: ...,3013,acute cholecystitis. laparoscopic cholecystect...


In [ ]:
testing_notes_with_smallest_size['sentences'] = testing_notes_with_smallest_size['text'].apply(process_clinical_note)
testing_notes_with_smallest_size

In [150]:


testing_notes_with_smallest_size['entities'] = testing_notes_with_smallest_size['sentences'].apply(extract_entities)
testing_notes_with_smallest_size

,hadm_id,text,text_length,sentences,entities
0,20019282,\nName: ___ Unit No: ___\n...,2480,Left ankle fracture. ORIF left ankle. yr old m...,"[left ankle fracture, or if left ankle, immedi..."
1,20022343,\nName: ___ Unit No: _...,3001,Preterm labor. repeat low transverse cesarean ...,"[pre term labor, po, ra pain, nt , oozing e..."
2,20025548,\nName: ___ Unit No: ___\n...,2961,contractions. SVD. Pt arrives co painful contr...,"[contraction s, sv d, painful contractions, ep..."
3,20027416,\nName: ___ Unit No: ___\n...,1059,neck cancer. direct laryngoscopy robotic hemig...,"[neck cancer i, head neck cancer]"
4,20028418,\nName: ___ Unit No: ___\n \...,1091,admission chemo. Pt tolerated desens without a...,"[acute adverse events, ovarian cancer]"
...,...,...,...,...,...
1010,29980814,\nName: ___ Unit No: _...,3329,Postpartum incision check. partial opening pac...,[partial opening packing incision partum cours...
1011,29980930,\nName: ___ Unit No: ___...,1094,elective chemotherapy. Pt presented elective c...,[]
1012,29987299,\nName: ___ Unit No: ___\...,2595,attach. BLOOD WBC RBC Hgb Hct. MCV MCH MCHC RD...,"[d, neu ts, mono s s, im abs neut, abs lymp ab..."
1013,29988390,\nName: ___ Unit No: ...,3013,acute cholecystitis. laparoscopic cholecystect...,"[acute cholecystitis, hx cholelithiasis, cb b..."


In [181]:
# Apply the function to the 'sentences' column and create a new 'predictions' column
testing_notes_with_smallest_size['predictions'] = testing_notes_with_smallest_size.apply(lambda row: get_icd_scores(row['entities'], index, row.name), axis=1)

INFO:root:Processed 0 rows
INFO:root:Processed 1 rows
INFO:root:Processed 2 rows
INFO:root:Processed 3 rows
INFO:root:Processed 4 rows
INFO:root:Processed 5 rows
INFO:root:Processed 6 rows
INFO:root:Processed 7 rows
INFO:root:Processed 8 rows
INFO:root:Processed 9 rows
INFO:root:Processed 10 rows
INFO:root:Processed 11 rows
INFO:root:Processed 12 rows
INFO:root:Processed 13 rows
INFO:root:Processed 14 rows
INFO:root:Processed 15 rows
INFO:root:Processed 16 rows
INFO:root:Processed 17 rows
INFO:root:Processed 18 rows
INFO:root:Processed 19 rows
INFO:root:Processed 20 rows
INFO:root:Processed 21 rows
INFO:root:Processed 22 rows
INFO:root:Processed 23 rows
INFO:root:Processed 24 rows
INFO:root:Processed 25 rows
INFO:root:Processed 26 rows
INFO:root:Processed 27 rows
INFO:root:Processed 28 rows
INFO:root:Processed 29 rows
INFO:root:Processed 30 rows
INFO:root:Processed 31 rows
INFO:root:Processed 32 rows
INFO:root:Processed 33 rows
INFO:root:Processed 34 rows
INFO:root:Processed 35 rows
IN

In [182]:
testing_notes_with_smallest_size

,hadm_id,text,text_length,sentences,entities,predictions
0,20019282,\nName: ___ Unit No: ___\n...,2480,Left ankle fracture. ORIF left ankle. yr old m...,"[left ankle fracture, or if left ankle, immedi...","[{'icd_code': 'S82.', 'score': 0.752511561}, {..."
1,20022343,\nName: ___ Unit No: _...,3001,Preterm labor. repeat low transverse cesarean ...,"[pre term labor, po, ra pain, nt , oozing e...","[{'icd_code': 'O60.', 'score': 0.748537779}, {..."
2,20025548,\nName: ___ Unit No: ___\n...,2961,contractions. SVD. Pt arrives co painful contr...,"[contraction s, sv d, painful contractions, ep...","[{'icd_code': 'P94.', 'score': 0.391164601}, {..."
3,20027416,\nName: ___ Unit No: ___\n...,1059,neck cancer. direct laryngoscopy robotic hemig...,"[neck cancer i, head neck cancer]","[{'icd_code': 'C10.', 'score': 0.514294505}, {..."
4,20028418,\nName: ___ Unit No: ___\n \...,1091,admission chemo. Pt tolerated desens without a...,"[acute adverse events, ovarian cancer]","[{'icd_code': 'Y71.', 'score': 0.651475489}, {..."
...,...,...,...,...,...,...
1010,29980814,\nName: ___ Unit No: _...,3329,Postpartum incision check. partial opening pac...,[partial opening packing incision partum cours...,"[{'icd_code': 'S21.', 'score': 0.529080093}, {..."
1011,29980930,\nName: ___ Unit No: ___...,1094,elective chemotherapy. Pt presented elective c...,[],None
1012,29987299,\nName: ___ Unit No: ___\...,2595,attach. BLOOD WBC RBC Hgb Hct. MCV MCH MCHC RD...,"[d, neu ts, mono s s, im abs neut, abs lymp ab...","[{'icd_code': 'W15.', 'score': 0.447785109}, {..."
1013,29988390,\nName: ___ Unit No: ...,3013,acute cholecystitis. laparoscopic cholecystect...,"[acute cholecystitis, hx cholelithiasis, cb b...","[{'icd_code': 'K81.', 'score': 0.919016838}, {..."


In [542]:

# Create the resultant DataFrame
result_df = pd.DataFrame()

for index, row in testing_notes_with_smallest_size.iterrows():
    hadm_id = str(row['hadm_id'])
    predictions = row['predictions']

    # Dictionary to store the maximum score for each label within a row
    max_scores = {}
    
    if predictions:
        for prediction in predictions:
            label = prediction['icd_code']
            score = prediction['score']

            # Update the max score for the label if a higher score is found
            if label in max_scores:
                max_scores[label] = max(max_scores[label], score)
            else:
                max_scores[label] = score

        # Append the results to the DataFrame
        for label, max_score in max_scores.items():
            # Optional: Specify a threshold (e.g., 0.20) to filter predictions
            if max_score >= 0.60:
                result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)

# Optionally, you can sort the DataFrame by 'hadm_id' and 'Score' if needed
result_df.sort_values(by=['hadm_id', 'Score'], ascending=[True, False], inplace=True)


C:\Users\BILAL\AppData\Local\Temp\ipykernel_19552\1198544566.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_19552\1198544566.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipykernel_19552\1198544566.py:26: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append({'hadm_id': hadm_id, 'icd_10_predicted': label, 'Score': max_score}, ignore_index=True)
C:\Users\BILAL\AppData\Local\Temp\ipyker

In [543]:
# Define the reformat function
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3] + '.'
    return code

In [544]:
result_df['icd_10_predicted'] = result_df['icd_10_predicted'].apply(lambda x: reformat(x))
result_df

,hadm_id,icd_10_predicted,Score
0,20019282,S82.,0.752512
1,20022343,O60.,0.748538
3,20022343,O64.,0.656174
2,20022343,A66.,0.628004
5,20025548,R11.,0.808978
...,...,...,...
3467,29988390,K85.,0.628160
3469,29990058,O61.,0.806058
3470,29990058,J81.,0.730772
3472,29990058,I15.,0.643342


In [545]:
result_df.dtypes

hadm_id              object
icd_10_predicted     object
Score               float64
dtype: object

In [546]:
unique_icd_codes = set(final_notes_with_icd_codes['icd_code'])
len(unique_icd_codes)

769

In [547]:
result_df

,hadm_id,icd_10_predicted,Score
0,20019282,S82.,0.752512
1,20022343,O60.,0.748538
3,20022343,O64.,0.656174
2,20022343,A66.,0.628004
5,20025548,R11.,0.808978
...,...,...,...
3467,29988390,K85.,0.628160
3469,29990058,O61.,0.806058
3470,29990058,J81.,0.730772
3472,29990058,I15.,0.643342


In [548]:
# Step 3: Initialize TP, FP, FN, TN counts
counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}


In [549]:
final_notes_with_icd_codes['hadm_id'] = final_notes_with_icd_codes['hadm_id'].astype(str)

In [550]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%100 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = result_df[result_df['icd_10_predicted'] == icd_code]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    counts[icd_code] = (len(TP), len(FP), len(FN), TN)

100 codes are done!
200 codes are done!
300 codes are done!
400 codes are done!
500 codes are done!
600 codes are done!
700 codes are done!


In [551]:
counts

{'G58.': (0, 0, 2, 0),
 'K76.': (0, 0, 16, 0),
 'M48.': (0, 0, 14, 0),
 'L50.': (0, 0, 1, 0),
 'V59.': (0, 0, 2, 0),
 'E73.': (0, 0, 1, 0),
 'F39.': (0, 2, 3, 0),
 'D66.': (1, 2, 2, 0),
 'S72.': (6, 24, 5, 0),
 'K62.': (0, 0, 9, 0),
 'C84.': (0, 0, 2, 0),
 'W15.': (0, 27, 1, 0),
 'K91.': (0, 0, 17, 0),
 'G43.': (1, 2, 17, 0),
 'S93.': (0, 0, 3, 0),
 'Z53.': (0, 0, 13, 0),
 'D70.': (0, 0, 1, 0),
 'G57.': (0, 13, 2, 0),
 'R42.': (0, 21, 7, 0),
 'Z12.': (0, 0, 3, 0),
 'N94.': (0, 0, 1, 0),
 'X58.': (0, 0, 32, 0),
 'F05.': (0, 0, 7, 0),
 'H66.': (0, 0, 1, 0),
 'W19.': (0, 0, 20, 0),
 'M87.': (1, 2, 2, 0),
 'I50.': (5, 11, 34, 0),
 'R11.': (17, 107, 24, 0),
 'T83.': (0, 0, 1, 0),
 'C44.': (0, 0, 2, 0),
 'L01.': (1, 1, 0, 0),
 'O00.': (2, 2, 1, 0),
 'F40.': (0, 21, 1, 0),
 'Z72.': (0, 0, 19, 0),
 'J35.': (0, 0, 4, 0),
 'M24.': (0, 0, 7, 0),
 'J09.': (1, 2, 0, 0),
 'G70.': (1, 0, 2, 0),
 'F20.': (0, 0, 4, 0),
 'Z17.': (0, 0, 4, 0),
 'S76.': (0, 0, 2, 0),
 'I99.': (0, 0, 1, 0),
 'R39.': (0, 0,

In [552]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall


In [553]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(precisions.values()) / len(precisions)
macro_recall = sum(recalls.values()) / len(recalls)

In [554]:
macro_precision

0.12972437999370784

In [555]:
macro_recall

0.1034717291133588

In [556]:
# Calculate Micro Precision and Recall
total_TP = sum([counts[icd_code][0] for icd_code in unique_icd_codes])
total_FP = sum([counts[icd_code][1] for icd_code in unique_icd_codes])
total_FN = sum([counts[icd_code][2] for icd_code in unique_icd_codes])

# Handle division by zero
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

Micro Precision: 0.2111987974445697
Micro Recall: 0.07086117765729416


In [557]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [558]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(f1_scores.values()) / len(f1_scores)

macro_f1_score

0.0888846079384624

In [559]:
# Handle division by zero for precision and recall in Micro F1 Score
if micro_precision + micro_recall == 0:
    micro_f1_score = 0
else:
    micro_f1_score = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

print("Micro F1 Score:", micro_f1_score)

Micro F1 Score: 0.10611782477341389


In [560]:
# Calculate Precision and Recall for Each ICD-10 Code
precisions = {}
recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    precisions[icd_code] = precision
    recalls[icd_code] = recall

# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in precisions.values() if precision > 0])

In [561]:
average_precision

0.5250423590271651

In [562]:
average_recall = np.mean([recall for recall in recalls.values() if recall > 0])

In [563]:
average_recall

0.4187882088851204

In [564]:
# Calculate F1 Score for Each ICD-10 Code
f1_scores = {}
for icd_code in unique_icd_codes:
    precision = precisions[icd_code]
    recall = recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    f1_scores[icd_code] = f1_score


In [565]:
average_f1 = np.mean([f1_score for f1_score in f1_scores.values() if f1_score > 0])

In [566]:
average_f1

0.3597487552877769

#### Top 50 ICD codes identification

In [567]:
top_50_icd_codes = final_notes_with_icd_codes['icd_code'].value_counts().index[:50]
top_50_icd_codes

Index(['Y92.', 'O99.', 'I10.', 'E78.', 'Z87.', 'Z3A.', 'Z51.', 'Z85.', 'E11.',
       'Z79.', 'Z68.', 'F41.', 'K21.', 'F32.', 'Z66.', 'Z86.', 'E66.', 'I25.',
       'Z37.', 'E87.', 'J45.', 'I48.', 'E03.', 'F17.', 'G47.', 'Y83.', 'Z95.',
       'J96.', 'G93.', 'O26.', 'N17.', 'N18.', 'G89.', 'C78.', 'J44.', 'D64.',
       'R40.', 'O34.', 'Z80.', 'O09.', 'R11.', 'I50.', 'Z23.', 'S06.', 'M19.',
       'N40.', 'I12.', 'I95.', 'S62.', 'O70.'],
      dtype='object')

In [568]:
filtered_test_notes = final_notes_with_icd_codes[final_notes_with_icd_codes['icd_code'].isin(top_50_icd_codes)]

filtered_test_notes

,hadm_id,text,text_length,icd_code
3,20019282,\nName: ___ Unit No: ___\n...,2480,Y92.
7,20022343,\nName: ___ Unit No: _...,3001,O34.
9,20022343,\nName: ___ Unit No: _...,3001,Z3A.
10,20022343,\nName: ___ Unit No: _...,3001,Z37.
11,20025548,\nName: ___ Unit No: ___\n...,2961,O99.
...,...,...,...,...
8578,29990058,\nName: ___ Unit No: ___\n...,3051,Z3A.
8579,29990058,\nName: ___ Unit No: ___\n...,3051,Z37.
8581,29990058,\nName: ___ Unit No: ___\n...,3051,Z23.
8582,29990058,\nName: ___ Unit No: ___\n...,3051,Z86.


In [569]:
filtered_unique_hadm_notes = filtered_test_notes[['hadm_id', 'text']].drop_duplicates(subset='hadm_id')

In [570]:
filtered_unique_hadm_notes = filtered_unique_hadm_notes.reset_index(drop = True)
filtered_unique_hadm_notes

,hadm_id,text
0,20019282,\nName: ___ Unit No: ___\n...
1,20022343,\nName: ___ Unit No: _...
2,20025548,\nName: ___ Unit No: ___\n...
3,20027416,\nName: ___ Unit No: ___\n...
4,20028418,\nName: ___ Unit No: ___\n \...
...,...,...
929,29980814,\nName: ___ Unit No: _...
930,29980930,\nName: ___ Unit No: ___...
931,29987299,\nName: ___ Unit No: ___\...
932,29988390,\nName: ___ Unit No: ...


In [571]:
filtered_icd10_data = {
    'hadm_id': [],
    'icd10_codes': []
}

In [572]:
unique_icd_codes = set(filtered_test_notes['icd_code'])

In [573]:
# Step 3: Initialize TP, FP, FN, TN counts
filtered_counts = {icd_code: (0, 0, 0, 0) for icd_code in unique_icd_codes}



In [574]:
i = 0
for icd_code in unique_icd_codes:
    i+=1
    if i%10 == 0:
        print(f'{i} codes are done!')
    
    true_icd_codes = filtered_test_notes[filtered_test_notes['icd_code'] == icd_code]['hadm_id'].values
    predicted_icd_codes = result_df[result_df['icd_10_predicted'].apply(lambda x: icd_code in x)]['hadm_id'].values

    TP = np.intersect1d(true_icd_codes, predicted_icd_codes)
    FP = np.setdiff1d(predicted_icd_codes, true_icd_codes)
    FN = np.setdiff1d(true_icd_codes, predicted_icd_codes)
    TN = len(np.union1d(predicted_icd_codes, true_icd_codes)) -len(TP) -len(FP) -len(FN)


    filtered_counts[icd_code] = (len(TP), len(FP), len(FN), TN)

10 codes are done!
20 codes are done!
30 codes are done!
40 codes are done!
50 codes are done!


In [575]:
filtered_counts

{'G47.': (2, 5, 69, 0),
 'S06.': (6, 9, 21, 0),
 'Z85.': (0, 0, 111, 0),
 'S62.': (3, 1, 14, 0),
 'O99.': (0, 0, 130, 0),
 'E66.': (7, 7, 90, 0),
 'Z37.': (0, 0, 86, 0),
 'I95.': (1, 7, 33, 0),
 'E78.': (14, 1, 183, 0),
 'Z95.': (0, 0, 48, 0),
 'J45.': (13, 13, 67, 0),
 'Z87.': (0, 0, 186, 0),
 'I12.': (0, 8, 34, 0),
 'N17.': (13, 2, 40, 0),
 'F17.': (0, 0, 73, 0),
 'D64.': (1, 0, 44, 0),
 'J44.': (1, 1, 44, 0),
 'Y83.': (0, 1, 71, 0),
 'E11.': (4, 0, 94, 0),
 'Z3A.': (3, 0, 190, 0),
 'K21.': (1, 0, 103, 0),
 'N40.': (0, 0, 34, 0),
 'O34.': (0, 0, 42, 0),
 'I10.': (0, 0, 224, 0),
 'Z66.': (0, 0, 100, 0),
 'J96.': (0, 0, 55, 0),
 'Z68.': (0, 0, 109, 0),
 'I50.': (5, 11, 34, 0),
 'R11.': (17, 107, 24, 0),
 'M19.': (0, 0, 31, 0),
 'Z23.': (0, 0, 40, 0),
 'N18.': (11, 5, 41, 0),
 'Z79.': (0, 0, 122, 0),
 'Y92.': (0, 0, 245, 0),
 'I48.': (5, 3, 69, 0),
 'Z80.': (0, 0, 24, 0),
 'R40.': (0, 1, 20, 0),
 'I25.': (2, 4, 65, 0),
 'Z86.': (0, 0, 82, 0),
 'G89.': (0, 0, 51, 0),
 'Z51.': (0, 0, 133,

In [576]:

# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall

In [577]:
filtered_precisions

{'G47.': 0.2857142857142857,
 'S06.': 0.4,
 'Z85.': 0,
 'S62.': 0.75,
 'O99.': 0,
 'E66.': 0.5,
 'Z37.': 0,
 'I95.': 0.125,
 'E78.': 0.9333333333333333,
 'Z95.': 0,
 'J45.': 0.5,
 'Z87.': 0,
 'I12.': 0.0,
 'N17.': 0.8666666666666667,
 'F17.': 0,
 'D64.': 1.0,
 'J44.': 0.5,
 'Y83.': 0.0,
 'E11.': 1.0,
 'Z3A.': 1.0,
 'K21.': 1.0,
 'N40.': 0,
 'O34.': 0,
 'I10.': 0,
 'Z66.': 0,
 'J96.': 0,
 'Z68.': 0,
 'I50.': 0.3125,
 'R11.': 0.13709677419354838,
 'M19.': 0,
 'Z23.': 0,
 'N18.': 0.6875,
 'Z79.': 0,
 'Y92.': 0,
 'I48.': 0.625,
 'Z80.': 0,
 'R40.': 0.0,
 'I25.': 0.3333333333333333,
 'Z86.': 0,
 'G89.': 0,
 'Z51.': 0,
 'O26.': 0,
 'O09.': 0.0,
 'E03.': 1.0,
 'F32.': 0,
 'E87.': 0,
 'G93.': 0,
 'O70.': 0.0,
 'C78.': 0,
 'F41.': 1.0}

In [578]:
filtered_recalls

{'G47.': 0.028169014084507043,
 'S06.': 0.2222222222222222,
 'Z85.': 0.0,
 'S62.': 0.17647058823529413,
 'O99.': 0.0,
 'E66.': 0.07216494845360824,
 'Z37.': 0.0,
 'I95.': 0.029411764705882353,
 'E78.': 0.07106598984771574,
 'Z95.': 0.0,
 'J45.': 0.1625,
 'Z87.': 0.0,
 'I12.': 0.0,
 'N17.': 0.24528301886792453,
 'F17.': 0.0,
 'D64.': 0.022222222222222223,
 'J44.': 0.022222222222222223,
 'Y83.': 0.0,
 'E11.': 0.04081632653061224,
 'Z3A.': 0.015544041450777202,
 'K21.': 0.009615384615384616,
 'N40.': 0.0,
 'O34.': 0.0,
 'I10.': 0.0,
 'Z66.': 0.0,
 'J96.': 0.0,
 'Z68.': 0.0,
 'I50.': 0.1282051282051282,
 'R11.': 0.4146341463414634,
 'M19.': 0.0,
 'Z23.': 0.0,
 'N18.': 0.21153846153846154,
 'Z79.': 0.0,
 'Y92.': 0.0,
 'I48.': 0.06756756756756757,
 'Z80.': 0.0,
 'R40.': 0.0,
 'I25.': 0.029850746268656716,
 'Z86.': 0.0,
 'G89.': 0.0,
 'Z51.': 0.0,
 'O26.': 0.0,
 'O09.': 0.0,
 'E03.': 0.013513513513513514,
 'F32.': 0.0,
 'E87.': 0.0,
 'G93.': 0.0,
 'O70.': 0.0,
 'C78.': 0.0,
 'F41.': 0.0283018

In [579]:
# Step 6: Calculate Macro Precision and Macro Recall
macro_precision = sum(filtered_precisions.values()) / len(filtered_precisions)
macro_recall = sum(filtered_recalls.values()) / len(filtered_recalls)


In [580]:
macro_precision

0.25912288786482335

In [581]:
macro_recall

0.04022638387371233

In [582]:
# Calculate Micro Precision and Recall
total_TP = sum([filtered_counts[icd_code][0] for icd_code in unique_icd_codes])
total_FP = sum([filtered_counts[icd_code][1] for icd_code in unique_icd_codes])
total_FN = sum([filtered_counts[icd_code][2] for icd_code in unique_icd_codes])

# Handle division by zero
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

print("Micro Precision:", micro_precision)
print("Micro Recall:", micro_recall)

Micro Precision: 0.369281045751634
Micro Recall: 0.028819178780923233


In [583]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [584]:
# Step 8: Calculate Macro F1 Score
macro_f1_score = sum(filtered_f1_scores.values()) / len(filtered_f1_scores)

# Now you have macro precision, macro recall, and macro F1 score for your multi-label classification evaluation
macro_f1_score


0.05480312789963528

In [585]:
# Calculate Micro F1 Score
micro_precision = total_TP / (total_TP + total_FP) if (total_TP + total_FP) != 0 else 0
micro_recall = total_TP / (total_TP + total_FN) if (total_TP + total_FN) != 0 else 0

# Handle division by zero for precision and recall in Micro F1 Score
if micro_precision + micro_recall == 0:
    micro_f1_score = 0
else:
    micro_f1_score = 2 * (micro_precision * micro_recall) / (micro_precision + micro_recall)

print("Micro F1 Score:", micro_f1_score)

Micro F1 Score: 0.05346581499881713


In [586]:
# Calculate Precision and Recall for Each ICD-10 Code
filtered_precisions = {}
filtered_recalls = {}
for icd_code in unique_icd_codes:
    TP, FP, FN, TN = filtered_counts[icd_code]
    
    # Handle division by zero
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)

    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    
    filtered_precisions[icd_code] = precision
    filtered_recalls[icd_code] = recall    
    
    
    
    
    
# Filter out zero precision values and calculate average precision
average_precision = np.mean([precision for precision in filtered_precisions.values() if precision > 0])

In [587]:
average_precision

0.6478072196620583

In [588]:
average_recall = np.mean([recall for recall in filtered_recalls.values() if recall > 0])

In [589]:
average_recall

0.10056595968428084

In [590]:
# Calculate F1 Score for Each ICD-10 Code
filtered_f1_scores = {}
for icd_code in unique_icd_codes:
    precision = filtered_precisions[icd_code]
    recall = filtered_recalls[icd_code]

    # Handle division by zero for precision and recall
    if precision + recall == 0:
        f1_score = 0
    else:
        f1_score = 2 * (precision * recall) / (precision + recall)

    filtered_f1_scores[icd_code] = f1_score


In [591]:
average_f1 = np.mean([f1_score for f1_score in filtered_f1_scores.values() if f1_score > 0])

In [592]:
average_f1

0.1370078197490882

# EXAMPLES 

In [636]:
def reformat(code):
    code = ''.join(code.split('.'))
    code = code[:3]
    return code

In [638]:
categories_description = pd.read_csv('categories.csv')
# categories_description['icd_code'] = categories_description['icd_code'].apply(reformat)
categories_description

,icd_code,description
0,A00,Cholera
1,A01,Typhoid and paratyphoid fevers
2,A010,Typhoid fever
3,A011,Paratyphoid fever A
4,A012,Paratyphoid fever B
...,...,...
24623,Z99,"Dependence on enabling machines and devices, n..."
24624,Z991,Dependence on respirator
24625,Z992,Dependence on renal dialysis
24626,Z993,Dependence on wheelchair


In [665]:
def predict_codes_using_AlBERT(processed_example,  categories_description, index,  threshold = 0.4):
    print("extracting_entities")
    extracted_entities_example = extract_entities(processed_example)
    print("entities extracted")
    
    print("Predicting codes")
    predictions = get_icd_scores(extracted_entities_example, index)
    print("codes predicted")
    
    predictions = [
    {**entry, 'icd_code': reformat(entry['icd_code'])}
    for entry in predictions
]
    
    for data, sentence in zip(predictions, extracted_entities_example):
        # Add a new key-value pair to each dictionary
        data['support_sentence'] = sentence
    predicted_codes = [
    item for item in predictions if item['score'] >= threshold
    ]
    # return predicted_codes

    final_results = pd.merge(pd.DataFrame(predicted_codes), categories_description, how = 'left', left_on = 'icd_code', right_on = 'icd_code')
    # final_results.drop(columns = ['icd_code'], inplace = True)

    return final_results

In [666]:
example_1 = final_notes_with_icd_codes.iloc[50].text

In [667]:

print(final_notes_with_icd_codes.iloc[50].text)

 
Name:  ___                Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: NEUROLOGY
 
Allergies: 
Calcium Channel Blocking Agent Diltiazem Analogues
 
Attending: ___
 
Chief Complaint:
Stroke
 
Major Surgical or Invasive Procedure:
None
 
Brief Hospital Course:
___ yr F w/ hx of ESRD on HD, HTN, DMII, and prior stroke in 
___ w/ residual mild aphasia who was transferred from OSH due 
to new onset R sided weakness and profound aphasia concerning 
for stroke. Exam is consistent with a L MCA syndrome w/ pt seen
to have severe expressive aphasia, L gaze preference, and R 
sided weakness. NIHSS at OSH was 22, tPA was given at OSH at 
___ on ___. NIHSS on ___ was 18. Etiology could potentially 
cardioembolic, given recurrent stroke and proximal vasculature
appearing clear on affected side. Her 24 hour head CT showed 
large L MCA
infarct, with no hemorrhage. Based on the CTA would not have 
expected the entire te

In [599]:
final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == '20053836']

,hadm_id,text,text_length,icd_code
25,20053836,\nName: ___ Unit No: ___\n ...,2491,I63.
26,20053836,\nName: ___ Unit No: ___\n ...,2491,N18.
27,20053836,\nName: ___ Unit No: ___\n ...,2491,J96.
28,20053836,\nName: ___ Unit No: ___\n ...,2491,J18.
29,20053836,\nName: ___ Unit No: ___\n ...,2491,B48.
30,20053836,\nName: ___ Unit No: ___\n ...,2491,E11.
31,20053836,\nName: ___ Unit No: ___\n ...,2491,D68.
32,20053836,\nName: ___ Unit No: ___\n ...,2491,E11.
33,20053836,\nName: ___ Unit No: ___\n ...,2491,I12.
34,20053836,\nName: ___ Unit No: ___\n ...,2491,T85.


In [601]:
pd.merge(final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == '20053836'],icd_categories_description, how = 'inner', left_on = 'icd_code', right_on =icd_categories_description.icd_code )

,icd_code,hadm_id,text,text_length,icd_code_x,icd_code_y,description
0,I63.,20053836,\nName: ___ Unit No: ___\n ...,2491,I63.,I63.,Cerebral infarction
1,N18.,20053836,\nName: ___ Unit No: ___\n ...,2491,N18.,N18.,Chronic kidney disease (CKD)
2,J96.,20053836,\nName: ___ Unit No: ___\n ...,2491,J96.,J96.,"Respiratory failure, not elsewhere classified"
3,J18.,20053836,\nName: ___ Unit No: ___\n ...,2491,J18.,J18.,"Pneumonia, unspecified organism"
4,B48.,20053836,\nName: ___ Unit No: ___\n ...,2491,B48.,B48.,"Other mycoses, not elsewhere classified"
5,E11.,20053836,\nName: ___ Unit No: ___\n ...,2491,E11.,E11.,Type 2 diabetes mellitus
6,E11.,20053836,\nName: ___ Unit No: ___\n ...,2491,E11.,E11.,Type 2 diabetes mellitus
7,D68.,20053836,\nName: ___ Unit No: ___\n ...,2491,D68.,D68.,Other coagulation defects
8,I12.,20053836,\nName: ___ Unit No: ___\n ...,2491,I12.,I12.,Hypertensive chronic kidney disease
9,T85.,20053836,\nName: ___ Unit No: ___\n ...,2491,T85.,T85.,Complications of other internal prosthetic dev...


In [668]:
processed_example = process_clinical_note(example_1)
print(processed_example)

Stroke. yr F w hx ESRD HD HTN DMII prior stroke. w residual mild aphasia transferred OSH due. new onset R sided weakness profound aphasia concerning. stroke Exam consistent L MCA syndrome w pt seen. severe expressive aphasia L gaze preference R. sided weakness NIHSS OSH tPA given OSH. NIHSS Etiology could potentially. cardioembolic given recurrent stroke proximal vasculature. appearing clear affected side hour head CT showed. large L MCA. expected entire territory infarct. appeared relatively good distal reconstitution. fetal PCA L appeared supplying MCA. infarct quite large affecting. cortical motor areas subcortical fibers would expect. remain hemiplegic R exam developed. global aphasia hemiplegia R Goals care. discussed family made comfort measures. passed away. ESRD received HD renal consulted. management Per family request discontinued. goals shifted comfort measures. yeast peritoneal fluid yeast grew peritoneal fluid. culture started fluconazole ID. consulted discontinued made CM

In [669]:
predict_codes_using_AlBERT(processed_example, categories_description, index)

extracting_entities
entities extracted
Predicting codes
codes predicted


,icd_code,score,support_sentence,description
0,I63,0.692511,stroke,Cerebral infarction
1,A66,0.583028,,Yaws
2,A66,0.419875,w h x,Yaws
3,R80,0.549966,esrd,Proteinuria
4,I11,0.546682,hd htn,Hypertensive heart disease
5,E11,0.549038,dm ii,Type 2 diabetes mellitus
6,I69,0.440887,prior stroke,Sequelae of cerebrovascular disease
7,F70,0.421660,w residual mild aphasia,Mild intellectual disabilities
8,I40,0.410475,new onset r sided weakness,Acute myocarditis
9,F73,0.599550,profound aphasia,Profound intellectual disabilities


In [673]:
example_2 = final_notes_with_icd_codes.iloc[100].text
print(example_2)

 
Name:  ___                  Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: CARDIOTHORACIC
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___.
 
Chief Complaint:
acute respiratory failure
 
Major Surgical or Invasive Procedure:
bronchoscopy x3

 
History of Present Illness:
From OMR:
HISTORY OF PRESENT ___ former smoker (___) who 
presented with dry cough for the past six months. Evaluation for 
this included a chest CT ___ which showed a right upper lobe 
mass with associated paratracheal lymphadenopathy. She underwent 
EBUS/TBNA ___ that had path of the RUL mass return as 
keratinizing squamous cell ca. Level 4R and 7 were negative. Her 
PET-CT ___ noted an FDG avid (SUV 20) 5.5 x 6.5 cm right 
upper lobe mass abutting mediastinal and parietal pleura. She 
underwent right thoracotomy and right pneumonectomy 
___ by pseudomonas PNA and respiratory failure 
s/p trach/PEG (___). With

In [688]:
processed_example = process_clinical_note(example_2)
print(processed_example)

acute respiratory failure. bronchoscopy x. HISTORY PRESENT former smoker. presented dry cough past six months Evaluation. included chest CT showed right upper lobe. mass associated paratracheal lymphadenopathy underwent. EBUSTBNA path RUL mass return. keratinizing squamous cell ca Level R negative. PETCT noted FDG avid SUV x cm right. upper lobe mass abutting mediastinal parietal pleura. underwent right thoracotomy right pneumonectomy. pseudomonas PNA respiratory failure. sp trachPEG recurrent worsening left lung. consolidation treated PNA. rehab earlier today returned hospital acute. respiratory failure likely secondary morphine Ativan. administration. deceased. patient rehab prolonged. hospital stay sp pneumonectomy However hours. arriving rehab facility patient desaturated. FiO brought back hospital urgent. bronchoscopy Bronchoscopy ED negative mucus. plugging questionable malacia component pt. admitted TSICU underwent Lasix diuresis. weaned pressure support However. underwent emerg

In [693]:
predict_codes_using_AlBERT(processed_example, categories_description, index)

extracting_entities
entities extracted
Predicting codes
codes predicted


,icd_code,score,support_sentence,description
0,J80,0.677276,acute respiratory failure,Acute respiratory distress syndrome
1,R05,0.861671,dry cough,Cough
2,C34,0.470111,right upper lobe,Malignant neoplasm of bronchus and lung
3,E65,0.456412,mass associated,Localized adiposity
4,R59,0.580624,para tracheal lymphadenopathy,Enlarged lymph nodes
5,W15,0.470217,e bustbna path rul mass return,Fall from cliff
6,C38,0.677994,upper lobe mass abutting mediastinal parietal ...,"Malignant neoplasm of heart, mediastinum and p..."
7,A67,0.422407,pseudo monas pna,Pinta [carate]
8,I50,0.658690,respiratory failure,Heart failure
9,A71,0.657987,sp trachpeg,Trachoma


In [691]:
pd.merge(final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == '20101637'],icd_categories_description, how = 'inner', left_on = 'icd_code', right_on =icd_categories_description.icd_code )

,icd_code,hadm_id,text,text_length,icd_code_x,icd_code_y,description
0,C34.,20101637,\nName: ___ Unit No: ___\...,3118,C34.,C34.,Malignant neoplasm of bronchus and lung
1,J15.,20101637,\nName: ___ Unit No: ___\...,3118,J15.,J15.,"Bacterial pneumonia, not elsewhere classified"
2,J95.,20101637,\nName: ___ Unit No: ___\...,3118,J95.,J95.,Intraoperative and postprocedural complication...
3,T40.,20101637,\nName: ___ Unit No: ___\...,3118,T40.,T40.,"Poisoning by, adverse effect of and underdosin..."
4,J96.,20101637,\nName: ___ Unit No: ___\...,3118,J96.,J96.,"Respiratory failure, not elsewhere classified"
5,J96.,20101637,\nName: ___ Unit No: ___\...,3118,J96.,J96.,"Respiratory failure, not elsewhere classified"
6,Z99.,20101637,\nName: ___ Unit No: ___\...,3118,Z99.,Z99.,"Dependence on enabling machines and devices, n..."
7,N17.,20101637,\nName: ___ Unit No: ___\...,3118,N17.,N17.,Acute kidney failure
8,E87.,20101637,\nName: ___ Unit No: ___\...,3118,E87.,E87.,"Other disorders of fluid, electrolyte and acid..."
9,J90.,20101637,\nName: ___ Unit No: ___\...,3118,J90.,J90.,"Pleural effusion, not elsewhere classified"


In [694]:
final_notes_with_icd_codes.iloc[200]

hadm_id                                                 20182442
text            \nName:  ___                        Unit No: ...
text_length                                                 1957
icd_code                                                    I71.
Name: 200, dtype: object

In [675]:
example_3 = final_notes_with_icd_codes.iloc[200].text
print(example_3)

 
Name:  ___                        Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: SURGERY
 
Allergies: 
No Allergies/ADRs on File
 
Attending: ___.
 
Chief Complaint:
Rupture Aortic Anuerysm
 
Major Surgical or Invasive Procedure:
Endovascular Aortic Aneurysm Repair

 
History of Present Illness:
 This is a ___ male with a known thoracoabdominal
aortic aneurysm who presented with severe back pain and
hypertension.  Upon presentation to the ED, the patient was
complaining of severe pain with a blood pressure in the ___, and
the history of the known aneurysm was obtained.  Bedside
ultrasound confirmed the aneurysm with periaortic fluid. 
Vascular surgery was called stat.  On evaluation the patient was
alert, however unable to communicate as he was only ___
speaking.

 
Past Medical History:
unknown

 
Social History:
___
Family History:
n/a
 
Physical Exam:
VS: HR ___, SBP ___, satting 100% on non-rebreather

In [696]:
processed_example = process_clinical_note(example_3)
print(processed_example)

Rupture Aortic Anuerysm. Endovascular Aortic Aneurysm Repair. male known thoracoabdominal. aortic aneurysm presented severe back pain. hypertension Upon presentation ED patient. complaining severe pain blood pressure. history known aneurysm obtained Bedside. ultrasound confirmed aneurysm periaortic fluid. Vascular surgery called stat evaluation patient. alert however unable communicate. HR SBP satting nonrebreather. pale obvious distress alert interactive. breathing comfortably. RRR. mildly distended. cold clammy. expired. patient presented ED ruptured aortic aneurysm. taken CT scanner brought. endovascular repair decompressive laparotomy. case continued ongoing bleeding abdomen. requiring massive transfusion maintain blood pressure. despite multiple pressors Given poor prognosis ongoing. bleeding family bedside decision. made stop transfusing passed away shortly. Rupture Aortic Aneurysm


In [697]:
predict_codes_using_AlBERT(processed_example, categories_description, index)

extracting_entities
entities extracted
Predicting codes
codes predicted


,icd_code,score,support_sentence,description
0,I71,0.631575,rupture aortic anuerysm,Aortic aneurysm and dissection
1,F52,0.473117,male known,Sexual dysfunction not due to a substance or k...
2,I72,0.720444,a ortic aneurysm,Other aneurysm
3,R10,0.664953,severe back pain,Abdominal and pelvic pain
4,I15,0.859660,hypertension,Secondary hypertension
5,R03,0.554766,complaining severe pain blood pressure,"Abnormal blood-pressure reading, without diagn..."
6,A66,0.554167,a ysm,Yaws
7,A66,0.583028,,Yaws
8,I15,0.553898,sbp,Secondary hypertension
9,A66,0.496464,sat ting,Yaws


In [695]:
pd.merge(final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == '20182442'],icd_categories_description, how = 'inner', left_on = 'icd_code', right_on =icd_categories_description.icd_code )

,icd_code,hadm_id,text,text_length,icd_code_x,icd_code_y,description
0,I71.,20182442,\nName: ___ Unit No: ...,1957,I71.,I71.,Aortic aneurysm and dissection
1,D62.,20182442,\nName: ___ Unit No: ...,1957,D62.,D62.,Acute posthemorrhagic anemia
2,I77.,20182442,\nName: ___ Unit No: ...,1957,I77.,I77.,Other disorders of arteries and arterioles
3,D68.,20182442,\nName: ___ Unit No: ...,1957,D68.,D68.,Other coagulation defects
4,E87.,20182442,\nName: ___ Unit No: ...,1957,E87.,E87.,"Other disorders of fluid, electrolyte and acid..."
5,E87.,20182442,\nName: ___ Unit No: ...,1957,E87.,E87.,"Other disorders of fluid, electrolyte and acid..."
6,I95.,20182442,\nName: ___ Unit No: ...,1957,I95.,I95.,Hypotension
7,Z66.,20182442,\nName: ___ Unit No: ...,1957,Z66.,Z66.,Do not resuscitate
8,N18.,20182442,\nName: ___ Unit No: ...,1957,N18.,N18.,Chronic kidney disease (CKD)
9,R68.,20182442,\nName: ___ Unit No: ...,1957,R68.,R68.,Other general symptoms and signs


In [677]:
example_4 = final_notes_with_icd_codes.iloc[300].text
print(example_4)

 
Name:  ___                       Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   M
 
Service: ORTHOPAEDICS
 
Allergies: 
Penicillins
 
Attending: ___.
 
Chief Complaint:
Left midfoot arthritis & forefoot deformity
 
Major Surgical or Invasive Procedure:
L Strayer, L midfoot fusion, L ___ MTP fusion, L ___ 
hammertoe corrections 
 
History of Present Illness:
___ w/ left midfoot arthritis & forefoot deformity
 
Social History:
___
Family History:
___
 
Physical Exam:
Splint clean dry intact
Moving all toes
SILT distally
Toes WWP
 
Brief Hospital Course:
___ s/p left midfoot fusion & forefoot deformity correction 
admitted for ___ & pain control. Uneventful hospital course. Home 
meds continued. No issues on day of discharge.
 
Medications on Admission:
The Preadmission Medication list is accurate and complete.
1. Lisinopril 10 mg PO DAILY 
2. BuPROPion XL (Once Daily) 300 mg PO DAILY 
3. Dolutegravir 50 mg PO DAILY 
4. 

In [698]:
processed_example = process_clinical_note(example_4)
print(processed_example)

Left midfoot arthritis forefoot deformity. L Strayer L midfoot fusion L MTP fusion L. hammertoe corrections. w left midfoot arthritis forefoot deformity. Splint clean dry intact. Moving toes. SILT distally. Toes WWP. sp left midfoot fusion forefoot deformity correction. admitted pain control Uneventful hospital course Home. Left foot deformity


In [699]:
predict_codes_using_AlBERT(processed_example, categories_description, index)

extracting_entities
entities extracted
Predicting codes
codes predicted


,icd_code,score,support_sentence,description
0,M76,0.598997,left midfoot arthritis,"Enthesopathies, lower limb, excluding foot"
1,Q66,0.537843,for efoot deformity,Congenital deformities of feet
2,M76,0.598997,left midfoot arthritis,"Enthesopathies, lower limb, excluding foot"
3,Q66,0.537843,for efoot deformity,Congenital deformities of feet
4,W15,0.518658,silt,Fall from cliff
5,Z43,0.487086,to es,Encounter for attention to artificial openings
6,A66,0.650415,w wp,Yaws
7,S92,0.447636,sp left midfoot,"Fracture of foot and toe, except ankle"
8,Q65,0.520620,for efoot deformity correction,Congenital deformities of hip
9,Q66,0.620191,left foot deformity,Congenital deformities of feet


In [701]:
pd.merge(final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == '20309437'],icd_categories_description, how = 'inner', left_on = 'icd_code', right_on =icd_categories_description.icd_code )

,icd_code,hadm_id,text,text_length,icd_code_x,icd_code_y,description
0,M13.,20309437,\nName: ___ Unit No: ...,2605,M13.,M13.,Other arthritis
1,M20.,20309437,\nName: ___ Unit No: ...,2605,M20.,M20.,Acquired deformities of fingers and toes
2,M20.,20309437,\nName: ___ Unit No: ...,2605,M20.,M20.,Acquired deformities of fingers and toes
3,M20.,20309437,\nName: ___ Unit No: ...,2605,M20.,M20.,Acquired deformities of fingers and toes
4,M20.,20309437,\nName: ___ Unit No: ...,2605,M20.,M20.,Acquired deformities of fingers and toes
5,M62.,20309437,\nName: ___ Unit No: ...,2605,M62.,M62.,Other disorders of muscle
6,G47.,20309437,\nName: ___ Unit No: ...,2605,G47.,G47.,Sleep disorders
7,B19.,20309437,\nName: ___ Unit No: ...,2605,B19.,B19.,Unspecified viral hepatitis
8,K21.,20309437,\nName: ___ Unit No: ...,2605,K21.,K21.,Gastro-esophageal reflux disease
9,D56.,20309437,\nName: ___ Unit No: ...,2605,D56.,D56.,Thalassemia


In [678]:
example_5 = final_notes_with_icd_codes.iloc[400].text
print(example_5)

 
Name:  ___                   Unit No:   ___
 
Admission Date:  ___              Discharge Date:   ___
 
Date of Birth:  ___             Sex:   F
 
Service: MEDICINE
 
Allergies: 
No Known Allergies / Adverse Drug Reactions
 
Attending: ___
 
Chief Complaint:
Inpatient Hospice
 
Major Surgical or Invasive Procedure:
none

 
History of Present Illness:
Patient was admitted to inpatient hospice at ___. For full H&P 
see prior discharge summary or admission note.
 
Past Medical History:
EtOH cirrhosis  
Calculus of gallbladder without cholecystitis  
COPD  
Fatty liver  
Iron deficiency anemia  
h/o aspiration pneumonitis  

Past Surgical History:
s/p partial gastrectomy ___ for ulcers
"revision" of gastric ulcer surgery in ___ 

 
Social History:
___
Family History:
Mother w/ DM and ESRD on HD  
No family members with liver disease. 

 
Physical Exam:
Admission to inpatient hospice exam:
Patient lying comfortably in bed, sleeping. Breathing unlabored.

Death exam:
Patient not breathing.

In [703]:
processed_example = process_clinical_note(example_5)
print(processed_example)

Inpatient Hospice. Patient admitted inpatient hospice full HP. see prior summary admission note. Past Surgical. sp partial gastrectomy ulcers. revision gastric ulcer surgery. Admission inpatient hospice. Patient lying comfortably bed sleeping Breathing unlabored. Death. pressure. Patient admitted inpatient hospice Pain agitation. medications titrated Medications manage secretions. added Patient expired. HYDROmorphone Dilaudid mg IV PRN. moderatesevere pain respiratory distress. LORazepam mg IV PRN anxiety nausea first line. Midodrine mg PO QH. Ondansetron mg IV PRN NauseaVomiting Second Line. Polyethylene Glycol g PO PRN Constipation First. Line. rifAXIMin mg PO BID. Simethicone mg PO PRN bloating. Sodium Chloride mL IV DAILY PRN line. Sodium Chloride mL IV DAILY PRN line. Renal failure. Liver failure. VRE bloodstream infection. Spontaneous bacterial peritonitis. Alcoholic cirrhosis


In [704]:
predict_codes_using_AlBERT(processed_example, categories_description, index)

extracting_entities
entities extracted
Predicting codes
codes predicted


,icd_code,score,support_sentence,description
0,K25,0.664114,partial gastrectomy ulcers,Gastric ulcer
1,P95,0.612906,death,Stillbirth
2,T70,0.599806,pressure,Effects of air pressure and water pressure
3,R52,0.449604,pain agitation,"Pain, unspecified"
4,R10,0.548361,moderate severe pain,Abdominal and pelvic pain
5,J80,0.788797,respiratory distress,Acute respiratory distress syndrome
6,R11,0.715194,anxiety nausea,Nausea and vomiting
7,R11,0.879423,nausea,Nausea and vomiting
8,W15,0.400164,cons tipation,Fall from cliff
9,W51,0.449862,bloating,Accidental striking against or bumped into by ...


In [705]:
pd.merge(final_notes_with_icd_codes[final_notes_with_icd_codes.hadm_id == '20407074'],icd_categories_description, how = 'inner', left_on = 'icd_code', right_on =icd_categories_description.icd_code )

,icd_code,hadm_id,text,text_length,icd_code_x,icd_code_y,description
0,Z51.,20407074,\nName: ___ Unit No: ___...,2820,Z51.,Z51.,Encounter for other aftercare and medical care
1,K65.,20407074,\nName: ___ Unit No: ___...,2820,K65.,K65.,Peritonitis
2,N17.,20407074,\nName: ___ Unit No: ___...,2820,N17.,N17.,Acute kidney failure
3,J96.,20407074,\nName: ___ Unit No: ___...,2820,J96.,J96.,"Respiratory failure, not elsewhere classified"
4,T80.,20407074,\nName: ___ Unit No: ___...,2820,T80.,T80.,"Complications following infusion, transfusion ..."
5,E87.,20407074,\nName: ___ Unit No: ___...,2820,E87.,E87.,"Other disorders of fluid, electrolyte and acid..."
6,E22.,20407074,\nName: ___ Unit No: ___...,2820,E22.,E22.,Hyperfunction of pituitary gland
7,K76.,20407074,\nName: ___ Unit No: ___...,2820,K76.,K76.,Other diseases of liver
8,D68.,20407074,\nName: ___ Unit No: ___...,2820,D68.,D68.,Other coagulation defects
9,N39.,20407074,\nName: ___ Unit No: ___...,2820,N39.,N39.,Other disorders of urinary system
